### --- Day 20: Pulse Propagation ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/20

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 20);
WriteLines(inputLines);

Loading puzzle file: Day20.txt
Total lines: 58
Max line length: 33

%db -> cq
%rj -> gp, nd
%ff -> bk
%rc -> gp
%bk -> tv


In [4]:
const bool LowPulse = false;
const bool HighPulse = true;

record Pulse(string source, bool pulse, string dest);

abstract class PulseModule(string key, string[] sources, string[] destinations) {
    protected string key = key;
    protected string[] sources = sources;
    public string[] destinations = destinations; // public for part 2

    public abstract IEnumerable<Pulse> Process(Pulse input);

    public override string ToString() => (GetType().Name, key, string.Join(',', sources), string.Join(',', destinations)).ToString();

    public string Key => key;
}

In [5]:
// Flip-flop modules (prefix %) are either on or off; they are initially off. If
// a flip-flop module receives a high pulse, it is ignored and nothing happens.
// However, if a flip-flop module receives a low pulse, it flips between on and
// off. If it was off, it turns on and sends a high pulse. If it was on, it turns
// off and sends a low pulse.

class FlipFlop(string key, string[] sources, string[] destinations) : PulseModule(key, sources, destinations)
{
    private bool OnOff = false; // off
    private Pulse[] Nothing = [];

    public override IEnumerable<Pulse> Process(Pulse input) {
        if (input.pulse == HighPulse) {
            return Nothing;
        }

        OnOff = !OnOff;

        return destinations.Select(d => new Pulse(key, OnOff, d)).ToList();
    }
}

In [6]:
// Conjunction modules (prefix &) remember the type of the most recent pulse
// received from each of their connected input modules; they initially default to
// remembering a low pulse for each input. When a pulse is received, the
// conjunction module first updates its memory for that input. Then, if it
// remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends
// a high pulse.

class Conjunction(string key, string[] sources, string[] destinations) : PulseModule(key, sources, destinations)
{
    private Dictionary<string, bool> pulseMemory = sources.ToDictionary(s => s, s => LowPulse);

    public override IEnumerable<Pulse> Process(Pulse input) {
        pulseMemory[input.source] = input.pulse;

        var allHigh = pulseMemory.Values.All(mem => mem == HighPulse);

        return destinations.Select(d => new Pulse(key, !allHigh, d));
    }
}

In [7]:
// There is a single broadcast module (named broadcaster). When it receives a
// pulse, it sends the same pulse to all of its destination modules.

class Broadcaster(string key, string[] sources, string[] destinations) : PulseModule(key, sources, destinations) 
{
    public override IEnumerable<Pulse> Process(Pulse input) => destinations.Select(d => new Pulse(key, input.pulse, d)).ToList();    
}

In [8]:
using ParsedModule = (string type, string key, string[] destinations);

IList<PulseModule> BuildModules(IEnumerable<ParsedModule> parsedModules) {
    var destLookup = parsedModules.ToDictionary(p => p.key, p => p.destinations);
    var sourcesLookup = destLookup.SelectMany(kv => kv.Value.Select(dest => (source: kv.Key, dest)))
                                .GroupBy(sd => sd.dest)
                                .ToDictionary(g => g.Key, g => g.Select(sd => sd.source).ToArray());
    

    PulseModule buildFunc(ParsedModule pm) {
        var sources = sourcesLookup.GetValueOrDefault(pm.key, []); // Can have no source
        var dests = destLookup[pm.key];

        return pm.type switch {
            "broadcaster" => new Broadcaster(pm.key, sources, dests),
            "%" => new FlipFlop(pm.key, sources, dests),
            "&" => new Conjunction(pm.key, sources, dests),
            _ => throw new Exception("Unexpected module type")
        };
    }

    return parsedModules.Select(buildFunc).ToList();
}

In [9]:
ParsedModule Parse(string line) {
    var lineBits = line.Split(" -> ");

    var dests = lineBits[1].Split(", ");

    var key = lineBits[0].Substring(1, lineBits[0].Length - 1);

    return lineBits[0][0] switch {
        'b' => ("broadcaster", "broadcaster", dests),
        '%' => ("%", key, dests),
        '&' => ("&", key, dests),
        _ => throw new Exception("Unrecognised module type")
    };
}

In [10]:
string[] testInputLines = [
    "broadcaster -> a, b, c",
    "%a -> b",
    "%b -> c",
    "%c -> inv",
    "&inv -> a",
];

var testInputParsedModules = testInputLines.Select(Parse);
var testInputModules = BuildModules(testInputParsedModules);
foreach (var x in testInputModules) {
    Console.WriteLine(x);
}

(Broadcaster, broadcaster, , a,b,c)
(FlipFlop, a, broadcaster,inv, b)
(FlipFlop, b, broadcaster,a, c)
(FlipFlop, c, broadcaster,b, inv)
(Conjunction, inv, c, a)


In [11]:
IEnumerable<Pulse> PressButton(IList<PulseModule> modules) 
{
    Queue<Pulse> pulses = new();
    var moduleDict = modules.ToDictionary(m => m.Key);

    pulses.Enqueue(new("button", LowPulse, "broadcaster"));

    var safety = 0;
    while (pulses.Count > 0) {
        var nextPulse = pulses.Dequeue();
        yield return nextPulse;

        // Need to account for this weirdness:
        //
        // &vf -> rx

        if (moduleDict.TryGetValue(nextPulse.dest, out var nextModule)) {
            var nextOutputs = nextModule.Process(nextPulse);
            foreach (var n in nextOutputs) {
                pulses.Enqueue(n); // no range?
            }
        }

        if (safety++ > 1000) {
            throw new Exception("Safety limit exceeded");
        }
    }
}

// button -low-> broadcaster
// broadcaster -low-> a
// broadcaster -low-> b
// broadcaster -low-> c
// a -high-> b
// b -high-> c
// c -high-> inv
// inv -low-> a
// a -low-> b
// b -low-> c
// c -low-> inv
// inv -high-> a

foreach (var p in PressButton(testInputModules)) {
    Console.WriteLine(p);
}

Pulse { source = button, pulse = False, dest = broadcaster }
Pulse { source = broadcaster, pulse = False, dest = a }
Pulse { source = broadcaster, pulse = False, dest = b }
Pulse { source = broadcaster, pulse = False, dest = c }
Pulse { source = a, pulse = True, dest = b }
Pulse { source = b, pulse = True, dest = c }
Pulse { source = c, pulse = True, dest = inv }
Pulse { source = inv, pulse = False, dest = a }
Pulse { source = a, pulse = False, dest = b }
Pulse { source = b, pulse = False, dest = c }
Pulse { source = c, pulse = False, dest = inv }
Pulse { source = inv, pulse = True, dest = a }


In [12]:
long CountPulses (IList<PulseModule> pulseModules) {
    var allPulses = Enumerable.Range(0, 1000).SelectMany(i => PressButton(pulseModules));
    var pulseCounts = allPulses.GroupBy(p => p.pulse).ToDictionary(g => g.Key, g => g.LongCount());

    return pulseCounts[LowPulse] * pulseCounts[HighPulse];
}

// In the first example, the same thing happens every time the button is pushed:
// 8 low pulses and 4 high pulses are sent. So, after pushing the button 1000
// times, 8000 low pulses and 4000 high pulses are sent. Multiplying these together
// gives 32000000.

var testAnswer = CountPulses(testInputModules);
Console.WriteLine(testAnswer);

32000000


In [13]:
var inputParsedModules = inputLines.Select(Parse);
var inputModules = BuildModules(inputParsedModules);

In [14]:
// Consult your module configuration; determine the number of low pulses and
// high pulses that would be sent after pushing the button 1000 times, waiting for
// all pulses to be fully handled after each push of the button. What do you get if
// you multiply the total number of low pulses sent by the total number of high
// pulses sent?

var part1Answer = CountPulses(inputModules);
Console.WriteLine(part1Answer);

680278040


In [15]:
// 680278040 is correct!
Ensure(680278040, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/20

In [17]:
// Let's try this one, it's sure to not work, but let's just get closure :)

In [18]:
void PressUntilRx(IList<PulseModule> modules) {
    var xx = Enumerable.Range(0, 10000).SelectMany(i => PressButton(modules));

    foreach (var pulse in xx) {
        if (pulse.dest == "rx" && pulse.pulse == LowPulse) {
            Console.WriteLine(pulse);
            throw new Exception("We hit rx");
        }
    }
}
PressUntilRx(inputModules);

// No hits after 3 mins. Ok we can confirm it's not happening :)

In [19]:
// Ok, I had to take a hint on this one. The suggestion is to render the graph- let's try with Mermaid

using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;

In [20]:
var command = new SubmitCode("console.log(\"hi\");", "javascript");

await Kernel.Root.SendAsync(command);

hi

In [21]:
flowchart LR
    A --> B
    A --> C
    A --> D
    B --> C
    B --> D

In [22]:
string WriteMermaid(IList<PulseModule> modules) {
    var sb = new StringBuilder();
    sb.AppendLine("flowchart LR");

    foreach (var m in modules.Reverse()) {
        foreach (var d in m.destinations) {
            sb.AppendLine($"    {m.Key} --> {d}");
        }
    }

    return sb.ToString();
}

async Task ShowInMermaidAsync(string mermaidStr) {
    var showMermaid = new SubmitCode(mermaidStr, "mermaid");
    await Kernel.Root.SendAsync(showMermaid);
}

await ShowInMermaidAsync(WriteMermaid(inputModules));



In [23]:
void WriteMermaidModule(IList<PulseModule> modules, HashSet<string> visited, string moduleName, int pad, StringBuilder sb) {
    if (visited.Contains(moduleName))
        return;

    visited.Add(moduleName);

    var pm = modules.Where(m => m.Key == moduleName).SingleOrDefault();
    
    var type = pm?.GetType().Name ?? "NULL";
    sb.AppendLine($"    {moduleName}({moduleName} - {type})");

    if (pm is null) {
        return;
    }

    foreach (var child in pm.destinations) {
        sb.AppendLine($"    {moduleName} --> {child}");
        WriteMermaidModule(modules, visited, child, pad + 2, sb);
    }
}

var sbx = new StringBuilder();
sbx.AppendLine("flowchart LR");

WriteMermaidModule(inputModules, new(), "broadcaster", 0, sbx);

Console.WriteLine(sbx.ToString());

await ShowInMermaidAsync(sbx.ToString());

flowchart LR
    broadcaster(broadcaster - Broadcaster)
    broadcaster --> sh
    sh(sh - FlipFlop)
    sh --> rr
    rr(rr - FlipFlop)
    rr --> ls
    ls(ls - FlipFlop)
    ls --> sl
    sl(sl - FlipFlop)
    sl --> cz
    cz(cz - Conjunction)
    cz --> hc
    hc(hc - FlipFlop)
    hc --> vr
    vr(vr - FlipFlop)
    vr --> cz
    vr --> qd
    qd(qd - FlipFlop)
    qd --> cz
    qd --> sz
    sz(sz - FlipFlop)
    sz --> cz
    sz --> fz
    fz(fz - FlipFlop)
    fz --> dn
    dn(dn - FlipFlop)
    dn --> cz
    fz --> cz
    cz --> kz
    kz(kz - FlipFlop)
    kz --> mj
    mj(mj - FlipFlop)
    mj --> hc
    mj --> cz
    cz --> rr
    cz --> hf
    hf(hf - Conjunction)
    hf --> vf
    vf(vf - Conjunction)
    vf --> rx
    rx(rx - NULL)
    cz --> sh
    sl --> kz
    ls --> cz
    sh --> cz
    broadcaster --> nm
    nm(nm - FlipFlop)
    nm --> rt
    rt(rt - Conjunction)
    rt --> pk
    pk(pk - Conjunction)
    pk --> vf
    rt --> xl
    xl(xl - FlipFlop)
    xl --> hp

In [24]:
// Ok, let's see if we can find out when mk hits

void PressUntilFound(IList<PulseModule> modules, string expectedDest, bool expectedPulse) {
    int previousFind = 0;

    foreach (var i in Enumerable.Range(1, 100_000)) {
        var pulses = PressButton(modules);

        foreach (var pulse in pulses) {
            if (pulse.dest == expectedDest && pulse.pulse == expectedPulse) {
                Console.WriteLine($"We found a target pulse at press {i} ({i - previousFind})");
                Console.WriteLine(pulse);

                previousFind = i;
            }
        }
    }
}
var inputModules = BuildModules(inputParsedModules);
PressUntilFound(inputModules, "pm", LowPulse);

// mk = 3889
// pm = 3881
// pk = 4021
// hf = 4013

We found a target pulse at press 3881 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 7762 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 11643 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 15524 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 19405 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 23286 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 27167 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 31048 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 34929 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 38810 (3881)
Pulse { source = gp, pulse = False, dest = pm }
We found a target pulse at press 42691 (3881)
Pulse { source =

In [25]:
static ulong GCD(ulong a, ulong b) {
    if (b == 0) {
        return a;
    }

    return GCD(b, a % b);
}

In [26]:
ulong mk = 3889;
ulong pm = 3881;
ulong pk = 4021;
ulong hf = 4013;

var g = GCD(mk, hf);
Console.WriteLine(g);

1


In [27]:
// Ok, having tried them all, GCD is 1, therefore the only time they will all be equal is when they are full multiples of each other

var part2Answer = mk * pm * pk * hf;
Console.WriteLine(part2Answer);

243548140870057


In [28]:
// 243548140870057 is correct!
Ensure(243548140870057UL, part2Answer);